In [1]:
# Importing necessary Libraries
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
# from ipywidgets import

In [2]:
# Loading the dataset.
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

# EDA

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


# Distribution of Feature

In [8]:
movies.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [9]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [10]:
ratings['userId'].nunique()

668

In [11]:
movies['movieId'].nunique(), ratings['movieId'].nunique()

(10329, 10325)

In [12]:
movies['genres']=movies['genres'].str.split("|")

In [13]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [14]:
movies2=movies.explode('genres')

In [15]:
movies2.head(20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance


In [16]:
merged_info=pd.merge(ratings,movies2,on=['movieId'],how="inner")
print(merged_info.groupby('genres').agg({"title":"nunique",
                                         "rating":"mean"}).reset_index().rename(columns={"title":"unique_movies_count",
                                                                                                     "rating":"mean_rating"}))


                genres  unique_movies_count  mean_rating
0   (no genres listed)                    7     3.071429
1               Action                 1736     3.451450
2            Adventure                 1164     3.518027
3            Animation                  400     3.635350
4             Children                  540     3.439429
5               Comedy                 3513     3.420996
6                Crime                 1440     3.642392
7          Documentary                  415     3.643035
8                Drama                 5217     3.650266
9              Fantasy                  670     3.500459
10           Film-Noir                  195     3.913636
11              Horror                 1001     3.281097
12                IMAX                  152     3.641821
13             Musical                  409     3.571962
14             Mystery                  675     3.652043
15             Romance                 1788     3.544255
16              Sci-Fi         

In [17]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [18]:
movies2['genres'].nunique()

20

In [19]:
#Removing (no genres listed) from genres.
movies2=movies2[movies2['genres']!="(no genres listed)"]

In [20]:
movies2['genres'].nunique()

19

In [21]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

# 1. Popularity Based Recommendation

In [22]:
#User Defined Function to display topN movies to be recommended 

def TopNPopularMovies(genre,num_ratings_threshold,topN=5):
  #Calculating mean of ratings and number of ratings
  popularity_df=merged_info.groupby(['genres','title']).agg({'rating':["mean","size"]}).reset_index()
  popularity_df.columns=['genre','title','ratings_mean','ratings_counts']
  
  #Filtering desired genre and number of reviews threshold 
  topN_recommendations=popularity_df[(popularity_df['genre']==genre) & (popularity_df['ratings_counts']>=num_ratings_threshold)].sort_values(by=['ratings_mean'],ascending=False).head(topN)
  
  #Refactoring output
  topN_recommendations['S.No']=list(range(1,len(topN_recommendations)+1))
  topN_recommendations.index=range(len(topN_recommendations))
  topN_recommendations.columns=['Genre','Movie Title','Average Movie Rating','Num Reviews','S.No']
  return topN_recommendations[['S.No','Movie Title','Average Movie Rating','Num Reviews']]


In [23]:
#Test 1
genre="Action"
num_ratings_threshold=100
topN=10
TopNPopularMovies(genre=genre,
                  num_ratings_threshold=num_ratings_threshold,
                  topN=topN)

,S.No,Movie Title,Average Movie Rating,Num Reviews
0,1,"Matrix, The (1999)",4.264368,261
1,2,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
2,3,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
3,4,Inception (2010),4.189320,103
4,5,Star Wars: Episode IV - A New Hope (1977),4.188645,273
5,6,Fight Club (1999),4.188406,207
6,7,Blade Runner (1982),4.169872,156
7,8,"Princess Bride, The (1987)",4.163743,171
8,9,Aliens (1986),4.146497,157
9,10,"Dark Knight, The (2008)",4.141732,127


In [24]:
# Test 2
genre="Comedy"
num_ratings_threshold=50
topN=5
TopNPopularMovies(genre=genre,
                  num_ratings_threshold=num_ratings_threshold,
                  topN=topN)

,S.No,Movie Title,Average Movie Rating,Num Reviews
0,1,Monty Python and the Holy Grail (1975),4.301948,154
1,2,Fargo (1996),4.271144,201
2,3,Life Is Beautiful (La Vita è bella) (1997),4.253425,73
3,4,"Sting, The (1973)",4.207792,77
4,5,Annie Hall (1977),4.205882,68


# 2. Content Based Filtering

In [25]:
#Creating a string of genres in order to apply TF-IDF further
movies3=movies2.groupby("title").agg({"genres":lambda x : " ".join(list(x))}).reset_index()
movies3

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy
...,...,...
10315,loudQUIETloud: A Film About the Pixies (2006),Documentary
10316,xXx (2002),Action Crime Thriller
10317,xXx: State of the Union (2005),Action Crime Thriller
10318,¡Three Amigos! (1986),Comedy Western


In [26]:
#Running TF-IDF using n grams from 1 to 3, this can be experimented with as well 
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies3['genres'])
#Computing cosine similarity 
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [27]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [28]:
#User defined function to recommend similar movies
def recommendations_genre(movie_df,similarity_matrix,movie_title,topN=5):
    #Target movie title
    titles = movie_df['title']
    #Index of all other movies 
    indices = pd.Series(movie_df.index, index=movie_df['title'])
    #Index of target movie 
    index = indices[movie_title]
    #Generating cosine similarity scores
    cosine_similarity_scores = list(enumerate(similarity_matrix[index]))
    #Descending order sorting based on scores
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)
    #Selecting topN movies to be recommended ( it can contain the same movie as well, hence choosing topN+2)
    cosine_similarity_scores = cosine_similarity_scores[1:topN+2]
    #Extracting matched movies 
    matching_movies = [i[0] for i in cosine_similarity_scores]
    matches_df=movie_df.iloc[matching_movies]
    matches_df=matches_df[matches_df['title']!=movie_title]
    #Refactoring output 
    matches_df.rename(columns={'title':'Movie Title'},inplace=True)
    matches_df['S.No']=range(1,len(matches_df)+1)
    matches_df.index=range(len(matches_df))
    return matches_df[['S.No','Movie Title']].head(topN)

In [29]:
# Test 1
movie_title="Toy Story (1995)"
topN=10
recommendations_genre(movie_df=movies3,
                      similarity_matrix=cosine_sim,
                      movie_title=movie_title,
                      topN=topN)

,S.No,Movie Title
0,1,Antz (1998)
1,2,Asterix and the Vikings (Astérix et les Viking...
2,3,"Boxtrolls, The (2014)"
3,4,DuckTales: The Movie - Treasure of the Lost La...
4,5,"Emperor's New Groove, The (2000)"
5,6,"Monsters, Inc. (2001)"
6,7,Shrek the Third (2007)
7,8,"Tale of Despereaux, The (2008)"
8,9,Toy Story 2 (1999)
9,10,Turbo (2013)


In [30]:
#Test 2
movie_title="Avatar (2009)"
topN=10
recommendations_genre(movie_df=movies3,
                      similarity_matrix=cosine_sim,
                      movie_title=movie_title,
                      topN=topN)

,S.No,Movie Title
0,1,"Amazing Spider-Man, The (2012)"
1,2,"Avengers, The (2012)"
2,3,Captain America: The Winter Soldier (2014)
3,4,Ender's Game (2013)
4,5,Godzilla (2014)
5,6,"Hunger Games: Catching Fire, The (2013)"
6,7,John Carter (2012)
7,8,Oblivion (2013)
8,9,Pacific Rim (2013)
9,10,Spider-Man 2 (2004)


In [31]:
#Using pivot table to create a one hot encoded vector for all the movies 
movies4=movies2.pivot_table(index=['title'],columns=['genres']).fillna(0).applymap(lambda x: int(x>0)).reset_index()

movies4.index=range(0,len(movies4))
new_columns=['title']+list(map(lambda x : x[1] , movies4.columns))[1:]
movies4.columns=new_columns

#Generating new data frame representing binarized genres 
movies5=pd.merge(movies[['title','movieId']],movies4)
movies5.head()

,title,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,Toy Story (1995),1,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji (1995),2,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men (1995),3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Waiting to Exhale (1995),4,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,Father of the Bride Part II (1995),5,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#Generating new similarity metrics 
movie_genre_vector = movies5.loc[:,"Action":].values
cosine_sim2 = cosine_similarity(movie_genre_vector, movie_genre_vector)

In [33]:
#Testing the same function,but using different vectorized dataframe and similarity metrics: Try 1

movie_title="Toy Story (1995)"
topN=10
recommendations_genre(movie_df=movies5,
                      similarity_matrix=cosine_sim2,
                      movie_title=movie_title,
                      topN=topN)

,S.No,Movie Title
0,1,Antz (1998)
1,2,Toy Story 2 (1999)
2,3,"Adventures of Rocky and Bullwinkle, The (2000)"
3,4,"Emperor's New Groove, The (2000)"
4,5,"Monsters, Inc. (2001)"
5,6,DuckTales: The Movie - Treasure of the Lost La...
6,7,"Wild, The (2006)"
7,8,Shrek the Third (2007)
8,9,"Tale of Despereaux, The (2008)"
9,10,Asterix and the Vikings (Astérix et les Viking...


In [34]:
#Testing the same function,but using different vectorized dataframe and similarity metrics: Try 2

movie_title="Avatar (2009)"
topN=10
recommendations_genre(movie_df=movies5,
                      similarity_matrix=cosine_sim2,
                      movie_title=movie_title,
                      topN=topN)

,S.No,Movie Title
0,1,Spider-Man 2 (2004)
1,2,Superman Returns (2006)
2,3,Star Trek (2009)
3,4,Transformers: Revenge of the Fallen (2009)
4,5,Tron: Legacy (2010)
5,6,"Avengers, The (2012)"
6,7,John Carter (2012)
7,8,"Amazing Spider-Man, The (2012)"
8,9,Oblivion (2013)
9,10,Star Trek Into Darkness (2013)


# 3. Collaborative Based Recommendation

In [35]:
#Movies with IDs
movies6=movies5[['title','movieId']]
movies6.head()

,title,movieId
0,Toy Story (1995),1
1,Jumanji (1995),2
2,Grumpier Old Men (1995),3
3,Waiting to Exhale (1995),4
4,Father of the Bride Part II (1995),5


In [36]:
#Ratings
ratings.head()


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [37]:
#Function to fetch user history
def FetchUserHistory(userid):
  user_ratings=ratings[ratings['userId']==userid]
  user_history=pd.merge(user_ratings,movies6,how="inner",on="movieId").sort_values('movieId')
  return user_history

In [38]:
def GenerateSimilarUsersRecommendations(target_user_id,p,k,topN):

  #Fetching user history 
  user_history=FetchUserHistory(userid=target_user_id)

  #Finding all users who have seen similar movies
  similar_users = ratings[ratings['movieId'].isin(user_history['movieId'].tolist())]

  #Finding num of movies reviewed by candidate similar users 
  similar_uses_num_movie_reviewed=similar_users.groupby("userId").agg({"movieId":"nunique"}).reset_index().rename(columns={'movieId':'num_movies_reviewed'})\
                                                                                          .sort_values(by='num_movies_reviewed',ascending=False)
  #Removing target user ID from the similar user subset
  similar_uses_num_movie_reviewed=similar_uses_num_movie_reviewed[similar_uses_num_movie_reviewed['userId']!=target_user_id]

  #Movies Rated by Target user
  movies_rated_by_target_user=user_history['movieId'].nunique()

  #Min movies rated by candidate similar users : Criteria 1 : Should be atleast 50% ( p is set to 50)
  min_movies_rated_threshold=int(movies_rated_by_target_user*(p/100))

  #Filtered candidates of similar users 
  similar_uses_num_movie_reviewed=similar_uses_num_movie_reviewed[similar_uses_num_movie_reviewed['num_movies_reviewed']>=min_movies_rated_threshold]

  # Ratings Data for filtered candidates of similar users 
  bestN_similar_users=similar_users[similar_users['userId'].isin(list(set(similar_uses_num_movie_reviewed.userId)))]

  #Creating vector representation for target user
  user_vector=user_history.pivot(index='userId',columns=['movieId'],values=['rating'])
  user_vector.columns=[str(each[1]) for each in user_vector.columns]

  #Creating vector representation for candidate similar users
  bestN_vectors=bestN_similar_users.pivot(index='userId',columns=['movieId'],values=['rating']).fillna(0)
  bestN_vectors.columns=[str(each[1]) for each in bestN_vectors.columns]

  #Matching column names
  for each in set(user_vector.columns)-set(bestN_vectors.columns):
    bestN_vectors[each]=0.0

  bestN_vectors=bestN_vectors[user_vector.columns]

  #Finding Cosine Similarity Amongst all the candidate users
  similarity_df=pd.DataFrame(cosine_similarity(user_vector,bestN_vectors)).T
  similarity_df.index=bestN_vectors.index
  similarity_df.columns=['similarity_score']

  #Filtering out top K similar users 
  topK_user_ids=list(similarity_df.sort_values('similarity_score',ascending=False).index[:k])

  #Selecting ratings data of similar K users 
  similar_users_subset=ratings[(ratings['userId'].isin(topK_user_ids))]


  #Selecting movies other than the ones rated by our target user
  similar_users_subset=similar_users_subset[~similar_users_subset['movieId'].isin(user_history['movieId'])]

  #Sorting the movies based on best average ratings of similar users and choosing topN movies
  best_recommended_movies=similar_users_subset.groupby('movieId').mean()[['rating']].reset_index().sort_values('rating',ascending=False)[:topN]

  #Output topN recommendations 
  output_df=pd.merge(best_recommended_movies,movies6)
  output_df.columns=['Movie ID','Average Rating', "Movie Title"]
  output_df['S.No']=range(1,len(output_df)+1)

  #User History
  user_history.rename(columns={'title':"Movie Title"},inplace=True)
  user_history=user_history.sort_values('rating',ascending=False)
  user_history['S.No']=range(1,len(user_history)+1)

  return user_history[['S.No',"Movie Title"]],output_df[['S.No',"Movie Title"]]

In [39]:
#Test 1

target_user_id=2
p=90
k=100
topN=10

history,recommendations=GenerateSimilarUsersRecommendations(target_user_id=target_user_id, p=p, k=k, topN=topN)

In [40]:
history

,S.No,Movie Title
0,1,Toy Story (1995)
26,2,"Time to Kill, A (1996)"
4,3,Sense and Sensibility (1995)
7,4,Dead Man Walking (1995)
10,5,Mr. Holland's Opus (1995)
17,6,Fargo (1996)
15,7,"River Wild, The (1994)"
27,8,Willy Wonka & the Chocolate Factory (1971)
25,9,Phenomenon (1996)
24,10,"Nutty Professor, The (1996)"


In [41]:
recommendations

,S.No,Movie Title
0,1,Mother (1996)
1,2,Up Close and Personal (1996)
2,3,Ransom (1996)
3,4,"Rock, The (1996)"
4,5,"Birdcage, The (1996)"
5,6,Heat (1995)
6,7,Happy Gilmore (1996)
7,8,Down Periscope (1996)
8,9,Muppet Treasure Island (1996)
9,10,Sabrina (1995)


In [42]:
#Test 2
target_user_id=4
p=50
k=100
topN=10
history,recommendations=GenerateSimilarUsersRecommendations(target_user_id=target_user_id,
                                    p=p,
                                    k=k,
                                    topN=topN)

In [43]:
history

,S.No,Movie Title
123,1,M*A*S*H (a.k.a. MASH) (1970)
93,2,Tarzan (1999)
26,3,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
101,4,"Color Purple, The (1985)"
29,5,Love in the Afternoon (1957)
...,...,...
106,120,Dudley Do-Right (1999)
97,121,Mystery Men (1999)
73,122,"Big Lebowski, The (1998)"
96,123,Lake Placid (1999)


In [ ]:
Recommenda